In [1]:
import sklearn
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, BaggingRegressor
from sklearn.ensemble import RandomTreesEmbedding, HistGradientBoostingRegressor, StackingRegressor, VotingRegressor
from sklearn.multioutput import MultiOutputRegressor, RegressorChain
from sklearn.model_selection import train_test_split
import time
import pandas as pd
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor, RidgeCV, Ridge
from sklearn.linear_model import ElasticNet, ElasticNetCV, Lars, LarsCV, Lasso, LassoCV, LassoLars, LassoLarsCV
from sklearn.linear_model import MultiTaskElasticNet, MultiTaskElasticNetCV
from sklearn.linear_model import MultiTaskLasso, MultiTaskLassoCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid, ParameterSampler, RandomizedSearchCV
from sklearn.gaussian_process.kernels import RBF, RationalQuadratic
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler, KernelCenterer, RobustScaler
from sklearn.svm import SVR, LinearSVR, NuSVR
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
import joblib
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, max_error
import time
from keras_tuner.tuners import RandomSearch
import keras_tuner
from keras_tuner.tuners import Hyperband
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.gridspec import GridSpec
from sklearn.preprocessing import StandardScaler

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## REDUCE MAX AERR

In [4]:
reduce_index = []

In [5]:
df_all = pd.read_csv('z_score_3_data_all.csv')
x_all = df_all.drop(columns = ["V","α","β"])
y_all = df_all[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [6]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999813543439 
 Beta : 0.977405104213847 
 V : 0.9999997375615296 
MAE : 
 Alpha : 0.002604547261349629 
 Beta : 0.559077213592743 
 V : 0.0018585468821989413 
MSE : 
 Alpha : 2.4353795012336788e-05 
 Beta : 2.5045052507851886 
 V : 5.227615544605306e-05 
Max AERR : 
 V     0.084655
α     0.044422
β    15.077669
dtype: float64
Waktu yang dibutuhkan :  2.118449338277181


In [7]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_1 = df_all.drop(index = index_max_aerr)
print(len(df_all_clean_1))
reduce_index.append(index_max_aerr)
print(reduce_index)

6744
[2653]


### AFTER REDUCE 1

In [8]:
x_all = df_all_clean_1.drop(columns = ["V","α","β"])
y_all = df_all_clean_1[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [9]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999738997234 
 Beta : 0.9791201241157456 
 V : 0.999999767956747 
MAE : 
 Alpha : 0.0028822101289861 
 Beta : 0.5819017930689058 
 V : 0.0017477101223856695 
MSE : 
 Alpha : 3.334355840427031e-05 
 Beta : 2.3498636326644653 
 V : 4.594868125542621e-05 
Max AERR : 
 V     0.083621
α     0.068427
β    15.632787
dtype: float64
Waktu yang dibutuhkan :  2.0624476273854575


In [10]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_2 = df_all_clean_1.drop(index = index_max_aerr)
print(len(df_all_clean_2))

6743


### AFTER REDUCE 2

In [13]:
x_all = df_all_clean_2.drop(columns = ["V","α","β"])
y_all = df_all_clean_2[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [14]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999806192993 
 Beta : 0.9764682222640888 
 V : 0.9999997368467912 
MAE : 
 Alpha : 0.0027025711803142873 
 Beta : 0.5690155804467043 
 V : 0.0018537152299413478 
MSE : 
 Alpha : 2.4552661153461546e-05 
 Beta : 2.662847160541583 
 V : 5.2715460462389246e-05 
Max AERR : 
 V     0.073038
α     0.050397
β    14.499736
dtype: float64
Waktu yang dibutuhkan :  2.104402287801107


In [15]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_3 = df_all_clean_2.drop(index = index_max_aerr)
print(len(df_all_clean_3))
reduce_index.append(index_max_aerr)
print(reduce_index)

6742
[2653, 3955]


### AFTER REDUCE 3

In [16]:
x_all = df_all_clean_3.drop(columns = ["V","α","β"])
y_all = df_all_clean_3[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [17]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999669928907 
 Beta : 0.9731994100130393 
 V : 0.9999996648216718 
MAE : 
 Alpha : 0.0035149331806165677 
 Beta : 0.5860376092335584 
 V : 0.0020648284304906213 
MSE : 
 Alpha : 4.23178177750007e-05 
 Beta : 2.848416035841298 
 V : 6.781785683303794e-05 
Max AERR : 
 V     0.088037
α     0.063712
β    15.539371
dtype: float64
Waktu yang dibutuhkan :  2.1014964461326597


In [18]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_4 = df_all_clean_3.drop(index = index_max_aerr)
print(len(df_all_clean_4))

6741


### AFTER REDUCE 4

In [19]:
x_all = df_all_clean_4.drop(columns = ["V","α","β"])
y_all = df_all_clean_4[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [20]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999765442166 
 Beta : 0.9770015563307579 
 V : 0.9999996674556278 
MAE : 
 Alpha : 0.0025846035005702333 
 Beta : 0.5992313748606602 
 V : 0.0020828580283698086 
MSE : 
 Alpha : 3.0181304130251413e-05 
 Beta : 2.492106473224148 
 V : 6.728378035838795e-05 
Max AERR : 
 V     0.088122
α     0.065167
β    14.856357
dtype: float64
Waktu yang dibutuhkan :  2.0963802496592203


In [21]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_5 = df_all_clean_4.drop(index = index_max_aerr)
print(len(df_all_clean_5))

6740


### AFTER REDUCE 5

In [22]:
x_all = df_all_clean_5.drop(columns = ["V","α","β"])
y_all = df_all_clean_5[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [23]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999836814871 
 Beta : 0.9778258240157613 
 V : 0.9999997400534842 
MAE : 
 Alpha : 0.0025025270281960636 
 Beta : 0.5667498731563047 
 V : 0.0018424718599312408 
MSE : 
 Alpha : 2.026631802744071e-05 
 Beta : 2.4481241317597764 
 V : 5.266746916140483e-05 
Max AERR : 
 V     0.072832
α     0.041409
β    15.529450
dtype: float64
Waktu yang dibutuhkan :  2.11029136578242


In [24]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_6 = df_all_clean_5.drop(index = index_max_aerr)
print(len(df_all_clean_6))

6739


### AFTER REDUCE 6 

In [25]:
x_all = df_all_clean_6.drop(columns = ["V","α","β"])
y_all = df_all_clean_6[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [26]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999822307682 
 Beta : 0.9818477981444924 
 V : 0.999999687754212 
MAE : 
 Alpha : 0.002450614219054185 
 Beta : 0.5215853722250828 
 V : 0.0019511397819043585 
MSE : 
 Alpha : 2.2642606877234367e-05 
 Beta : 2.0241461146453985 
 V : 6.38578820067148e-05 
Max AERR : 
 V     0.081263
α     0.043083
β    13.504422
dtype: float64
Waktu yang dibutuhkan :  2.1017019629478453


In [27]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_7 = df_all_clean_6.drop(index = index_max_aerr)
print(len(df_all_clean_7))

6738


### AFTER REDUCE 7

In [28]:
x_all = df_all_clean_7.drop(columns = ["V","α","β"])
y_all = df_all_clean_7[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [29]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999793103639 
 Beta : 0.9789026590297131 
 V : 0.9999996413821204 
MAE : 
 Alpha : 0.002494045401630279 
 Beta : 0.5558277470708812 
 V : 0.0020110825643576582 
MSE : 
 Alpha : 2.654287289538122e-05 
 Beta : 2.3118250708688306 
 V : 7.196567480104904e-05 
Max AERR : 
 V     0.093326
α     0.046598
β    14.609380
dtype: float64
Waktu yang dibutuhkan :  2.076192589600881


In [30]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_8 = df_all_clean_7.drop(index = index_max_aerr)
print(len(df_all_clean_8))

6737


### AFTER REDUCE 8

In [31]:
x_all = df_all_clean_8.drop(columns = ["V","α","β"])
y_all = df_all_clean_8[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [32]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999781896386 
 Beta : 0.9801959168414832 
 V : 0.9999996464004162 
MAE : 
 Alpha : 0.002754751631910952 
 Beta : 0.5512583874067829 
 V : 0.002007195348630702 
MSE : 
 Alpha : 2.787414099307365e-05 
 Beta : 2.1839373000771434 
 V : 7.095727794432465e-05 
Max AERR : 
 V     0.083895
α     0.061856
β    12.531299
dtype: float64
Waktu yang dibutuhkan :  2.0541852513949075


In [33]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_9 = df_all_clean_8.drop(index = index_max_aerr)
print(len(df_all_clean_9))

6736


### AFTER REDUCE 9

In [34]:
x_all = df_all_clean_9.drop(columns = ["V","α","β"])
y_all = df_all_clean_9[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [35]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999847327692 
 Beta : 0.9828985978891092 
 V : 0.9999997588293317 
MAE : 
 Alpha : 0.0025779760378018017 
 Beta : 0.525255326822909 
 V : 0.0016775034990092299 
MSE : 
 Alpha : 1.9302606994699728e-05 
 Beta : 1.9081823124103434 
 V : 4.788564167092086e-05 
Max AERR : 
 V     0.081904
α     0.039391
β    13.293029
dtype: float64
Waktu yang dibutuhkan :  2.0292421142260233


In [36]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_10 = df_all_clean_9.drop(index = index_max_aerr)
print(len(df_all_clean_10))

6735


### AFTER REDUCE 10

In [37]:
x_all = df_all_clean_10.drop(columns = ["V","α","β"])
y_all = df_all_clean_10[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [38]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999847716097 
 Beta : 0.9823574375512504 
 V : 0.9999997476003271 
MAE : 
 Alpha : 0.002459508828171174 
 Beta : 0.538656684449404 
 V : 0.0017734393959564043 
MSE : 
 Alpha : 1.9526928835002922e-05 
 Beta : 1.9527844282548887 
 V : 5.0019906882486306e-05 
Max AERR : 
 V     0.093602
α     0.048514
β    10.960652
dtype: float64
Waktu yang dibutuhkan :  2.0263363043467204


In [39]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_11 = df_all_clean_10.drop(index = index_max_aerr)
print(len(df_all_clean_11))

6734


### AFTER REDUCE 11

In [40]:
x_all = df_all_clean_11.drop(columns = ["V","α","β"])
y_all = df_all_clean_11[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [41]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999762767131 
 Beta : 0.9821580404321636 
 V : 0.9999997195142062 
MAE : 
 Alpha : 0.0029681725512796828 
 Beta : 0.5716475210619505 
 V : 0.001685722997489279 
MSE : 
 Alpha : 2.9414987006612827e-05 
 Beta : 1.9656345299951392 
 V : 5.5094277108102305e-05 
Max AERR : 
 V     0.089227
α     0.054734
β    13.832811
dtype: float64
Waktu yang dibutuhkan :  1.9326043883959452


In [42]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_12 = df_all_clean_11.drop(index = index_max_aerr)
print(len(df_all_clean_12))

6733


### AFTER REDUCE 12

In [43]:
x_all = df_all_clean_12.drop(columns = ["V","α","β"])
y_all = df_all_clean_12[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [44]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999832227282 
 Beta : 0.9798538567233789 
 V : 0.9999996568547769 
MAE : 
 Alpha : 0.002460906974746163 
 Beta : 0.5900623046905906 
 V : 0.00213703640910925 
MSE : 
 Alpha : 2.155971270978009e-05 
 Beta : 2.214224376352329 
 V : 6.842071335097357e-05 
Max AERR : 
 V     0.084347
α     0.049333
β    13.882795
dtype: float64
Waktu yang dibutuhkan :  1.8329981207847594


In [45]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_13 = df_all_clean_12.drop(index = index_max_aerr)
print(len(df_all_clean_13))

6732


### AFTER REDUCE 13

In [46]:
x_all = df_all_clean_13.drop(columns = ["V","α","β"])
y_all = df_all_clean_13[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [47]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999770340473 
 Beta : 0.978895234081505 
 V : 0.9999996863784699 
MAE : 
 Alpha : 0.0027333968701645246 
 Beta : 0.579314496937547 
 V : 0.0019848761895335243 
MSE : 
 Alpha : 2.9445374186753157e-05 
 Beta : 2.283543770410889 
 V : 6.253012814416252e-05 
Max AERR : 
 V     0.084885
α     0.059408
β    17.980620
dtype: float64
Waktu yang dibutuhkan :  1.8530572295188903


In [48]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_14 = df_all_clean_13.drop(index = index_max_aerr)
print(len(df_all_clean_14))

6731


### AFTER REDUCE 14

In [49]:
x_all = df_all_clean_14.drop(columns = ["V","α","β"])
y_all = df_all_clean_14[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [50]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.999999971951516 
 Beta : 0.9835814539843236 
 V : 0.9999997189210671 
MAE : 
 Alpha : 0.0028919581193079576 
 Beta : 0.5193785366191195 
 V : 0.0017748315868983105 
MSE : 
 Alpha : 3.518949462385284e-05 
 Beta : 1.8010876468890926 
 V : 5.551747807891382e-05 
Max AERR : 
 V     0.072790
α     0.057305
β    14.930316
dtype: float64
Waktu yang dibutuhkan :  1.8993250211079915


In [51]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_15 = df_all_clean_14.drop(index = index_max_aerr)
print(len(df_all_clean_15))

6730


### AFTER REDUCE 15

In [52]:
x_all = df_all_clean_15.drop(columns = ["V","α","β"])
y_all = df_all_clean_15[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [53]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999767816471 
 Beta : 0.9863907164332788 
 V : 0.9999997357014233 
MAE : 
 Alpha : 0.0025771540855073116 
 Beta : 0.47403351967560065 
 V : 0.0017989483362961872 
MSE : 
 Alpha : 2.9228104938293327e-05 
 Beta : 1.494001177988985 
 V : 5.2465858281057316e-05 
Max AERR : 
 V     0.087740
α     0.060261
β    13.684451
dtype: float64
Waktu yang dibutuhkan :  1.8439833561579386


In [54]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_16 = df_all_clean_15.drop(index = index_max_aerr)
print(len(df_all_clean_16))

6729


### AFTER REDUCE 16

In [55]:
x_all = df_all_clean_16.drop(columns = ["V","α","β"])
y_all = df_all_clean_16[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [56]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999757291171 
 Beta : 0.9815450665187536 
 V : 0.9999997692467333 
MAE : 
 Alpha : 0.0029848262189479803 
 Beta : 0.5274674806752149 
 V : 0.0016795059978588621 
MSE : 
 Alpha : 3.0628085937382114e-05 
 Beta : 1.883304643582854 
 V : 4.6841386472197887e-05 
Max AERR : 
 V     0.084882
α     0.055166
β    12.851132
dtype: float64
Waktu yang dibutuhkan :  1.8400457382202149


In [57]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_17 = df_all_clean_16.drop(index = index_max_aerr)
print(len(df_all_clean_17))

6728


### AFTER REDUCE 17

In [58]:
x_all = df_all_clean_17.drop(columns = ["V","α","β"])
y_all = df_all_clean_17[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [59]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.999999986475118 
 Beta : 0.9776728087783741 
 V : 0.9999997299892933 
MAE : 
 Alpha : 0.0024857069837767925 
 Beta : 0.5809374231314947 
 V : 0.0018242004304755241 
MSE : 
 Alpha : 1.706560907384472e-05 
 Beta : 2.4433485604002474 
 V : 5.4057068597547857e-05 
Max AERR : 
 V     0.093124
α     0.035215
β    14.833616
dtype: float64
Waktu yang dibutuhkan :  1.8284637928009033


In [60]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_18 = df_all_clean_17.drop(index = index_max_aerr)
print(len(df_all_clean_18))

6727


### AFTER REDUCE 18

In [61]:
x_all = df_all_clean_18.drop(columns = ["V","α","β"])
y_all = df_all_clean_18[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [62]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999814556167 
 Beta : 0.9785753051319641 
 V : 0.9999997728580574 
MAE : 
 Alpha : 0.002567372827064675 
 Beta : 0.5568818818459476 
 V : 0.0016965544811241096 
MSE : 
 Alpha : 2.3438089826800093e-05 
 Beta : 2.30343059041354 
 V : 4.532341639275604e-05 
Max AERR : 
 V     0.065308
α     0.062579
β    13.671245
dtype: float64
Waktu yang dibutuhkan :  1.8411261995633443


In [63]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_19 = df_all_clean_18.drop(index = index_max_aerr)
print(len(df_all_clean_19))

6726


### AFTER REDUCE 19

In [64]:
x_all = df_all_clean_19.drop(columns = ["V","α","β"])
y_all = df_all_clean_19[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [65]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999846998079 
 Beta : 0.9794740105022052 
 V : 0.9999996938745352 
MAE : 
 Alpha : 0.002339195187601179 
 Beta : 0.5666528885071419 
 V : 0.0017232190815275853 
MSE : 
 Alpha : 1.9221953801128323e-05 
 Beta : 2.2562776879457083 
 V : 6.157395301992822e-05 
Max AERR : 
 V     0.130129
α     0.044276
β    11.438039
dtype: float64
Waktu yang dibutuhkan :  1.8345757881800333


In [66]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_20 = df_all_clean_19.drop(index = index_max_aerr)
print(len(df_all_clean_20))

6725


### AFTER REDUCE 20

In [67]:
x_all = df_all_clean_20.drop(columns = ["V","α","β"])
y_all = df_all_clean_20[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [68]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999784240549 
 Beta : 0.98083640212229 
 V : 0.9999997030138347 
MAE : 
 Alpha : 0.002665532971901297 
 Beta : 0.515688745911372 
 V : 0.002020582176182436 
MSE : 
 Alpha : 2.7133336846981392e-05 
 Beta : 2.0311342754457633 
 V : 5.8863213076458125e-05 
Max AERR : 
 V     0.075154
α     0.066106
β    13.997942
dtype: float64
Waktu yang dibutuhkan :  1.8358165542284648


In [69]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_21 = df_all_clean_20.drop(index = index_max_aerr)
print(len(df_all_clean_21))

6724


### AFTER REDUCE 21

In [70]:
x_all = df_all_clean_21.drop(columns = ["V","α","β"])
y_all = df_all_clean_21[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [71]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.999999982615092 
 Beta : 0.9834776440134287 
 V : 0.9999997491265673 
MAE : 
 Alpha : 0.0027604065001561187 
 Beta : 0.48633491791061023 
 V : 0.0018246195926608683 
MSE : 
 Alpha : 2.161358572749577e-05 
 Beta : 1.7783882996031715 
 V : 5.0067257958282746e-05 
Max AERR : 
 V    0.073124
α    0.036472
β    9.831914
dtype: float64
Waktu yang dibutuhkan :  1.8404672662417094


In [72]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_22 = df_all_clean_21.drop(index = index_max_aerr)
print(len(df_all_clean_22))

6723


### AFTER REDUCE 22

In [73]:
x_all = df_all_clean_22.drop(columns = ["V","α","β"])
y_all = df_all_clean_22[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [74]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999794197231 
 Beta : 0.9807853720516672 
 V : 0.9999997943596312 
MAE : 
 Alpha : 0.0025520050155960687 
 Beta : 0.544587359906684 
 V : 0.0016887194700017118 
MSE : 
 Alpha : 2.6079050099830286e-05 
 Beta : 2.2330498634008915 
 V : 4.1636377271704355e-05 
Max AERR : 
 V     0.065292
α     0.067731
β    12.807044
dtype: float64
Waktu yang dibutuhkan :  1.832316807905833


In [75]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_23 = df_all_clean_22.drop(index = index_max_aerr)
print(len(df_all_clean_23))

6722


### AFTER REDUCE 23

In [76]:
x_all = df_all_clean_23.drop(columns = ["V","α","β"])
y_all = df_all_clean_23[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [77]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999806132437 
 Beta : 0.9835708827220897 
 V : 0.9999997376480758 
MAE : 
 Alpha : 0.0026921736237225944 
 Beta : 0.48614890292291385 
 V : 0.0016406386290793791 
MSE : 
 Alpha : 2.4327883519287096e-05 
 Beta : 1.7144182681955114 
 V : 5.353701738872308e-05 
Max AERR : 
 V     0.129659
α     0.051815
β    11.856373
dtype: float64
Waktu yang dibutuhkan :  1.8368247906366983


In [78]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_24 = df_all_clean_23.drop(index = index_max_aerr)
print(len(df_all_clean_24))

6721


### AFTER REDUCE 24

In [79]:
x_all = df_all_clean_24.drop(columns = ["V","α","β"])
y_all = df_all_clean_24[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [80]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999829516517 
 Beta : 0.9843992864286789 
 V : 0.999999698385499 
MAE : 
 Alpha : 0.0025257070083602386 
 Beta : 0.5012392810581787 
 V : 0.0017172482587969004 
MSE : 
 Alpha : 2.1668526749261163e-05 
 Beta : 1.7158788513151138 
 V : 6.014411388032098e-05 
Max AERR : 
 V     0.145933
α     0.046801
β    12.039664
dtype: float64
Waktu yang dibutuhkan :  1.837827444076538


In [81]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_25 = df_all_clean_24.drop(index = index_max_aerr)
print(len(df_all_clean_25))

6720


### AFTER REDUCE 25

In [82]:
x_all = df_all_clean_25.drop(columns = ["V","α","β"])
y_all = df_all_clean_25[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [83]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999795754949 
 Beta : 0.9856145820117687 
 V : 0.9999997493919992 
MAE : 
 Alpha : 0.00264829970693345 
 Beta : 0.4973368439865245 
 V : 0.0014094500790359484 
MSE : 
 Alpha : 2.5335405613909085e-05 
 Beta : 1.6406328643111545 
 V : 5.0239081873899903e-05 
Max AERR : 
 V     0.130989
α     0.064427
β    10.373010
dtype: float64
Waktu yang dibutuhkan :  1.8444350481033325


In [84]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_26 = df_all_clean_25.drop(index = index_max_aerr)
print(len(df_all_clean_26))

6719


### AFTER REDUCE 26

In [85]:
x_all = df_all_clean_26.drop(columns = ["V","α","β"])
y_all = df_all_clean_26[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [86]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999739259854 
 Beta : 0.9861060148651558 
 V : 0.9999997121048094 
MAE : 
 Alpha : 0.002679255758099746 
 Beta : 0.48564560659717143 
 V : 0.0015824671151048926 
MSE : 
 Alpha : 3.2450318611178066e-05 
 Beta : 1.5799553023269302 
 V : 5.773442719540216e-05 
Max AERR : 
 V     0.129415
α     0.075332
β    10.881309
dtype: float64
Waktu yang dibutuhkan :  1.8543764154116313


In [87]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_27 = df_all_clean_26.drop(index = index_max_aerr)
print(len(df_all_clean_27))

6718


### AFTER REDUCE 27

In [88]:
x_all = df_all_clean_27.drop(columns = ["V","α","β"])
y_all = df_all_clean_27[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [89]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999712362085 
 Beta : 0.9858954162591556 
 V : 0.9999997255584597 
MAE : 
 Alpha : 0.003118920576860782 
 Beta : 0.473201973255956 
 V : 0.0016169976745297175 
MSE : 
 Alpha : 3.611172563054531e-05 
 Beta : 1.5607572758738537 
 V : 5.550466648433304e-05 
Max AERR : 
 V    0.128622
α    0.067817
β    9.963261
dtype: float64
Waktu yang dibutuhkan :  1.8327463150024415


In [90]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_28 = df_all_clean_27.drop(index = index_max_aerr)
print(len(df_all_clean_28))

6717


### AFTER REDUCE 28

In [91]:
x_all = df_all_clean_28.drop(columns = ["V","α","β"])
y_all = df_all_clean_28[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [92]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999805737761 
 Beta : 0.9841475348809093 
 V : 0.9999997183254854 
MAE : 
 Alpha : 0.0024650892663839095 
 Beta : 0.5207688319865559 
 V : 0.0017830227809993518 
MSE : 
 Alpha : 2.5118577657476363e-05 
 Beta : 1.802740973759886 
 V : 5.563998809747977e-05 
Max AERR : 
 V     0.081082
α     0.063207
β    12.102140
dtype: float64
Waktu yang dibutuhkan :  1.8356406887372334


In [93]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_29 = df_all_clean_28.drop(index = index_max_aerr)
print(len(df_all_clean_29))

6716


### AFTER REDUCE 29

In [94]:
x_all = df_all_clean_29.drop(columns = ["V","α","β"])
y_all = df_all_clean_29[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [95]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999774747178 
 Beta : 0.984840596403425 
 V : 0.9999997365237846 
MAE : 
 Alpha : 0.002849919398260284 
 Beta : 0.506690538294264 
 V : 0.001868202676855257 
MSE : 
 Alpha : 2.8688724071377963e-05 
 Beta : 1.700739692789042 
 V : 5.16202071973203e-05 
Max AERR : 
 V     0.065221
α     0.057853
β    10.554475
dtype: float64
Waktu yang dibutuhkan :  1.8505324204762776


In [96]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_30 = df_all_clean_29.drop(index = index_max_aerr)
print(len(df_all_clean_30))

6715


### AFTER REDUCE 30

In [97]:
x_all = df_all_clean_30.drop(columns = ["V","α","β"])
y_all = df_all_clean_30[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [98]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999727602648 
 Beta : 0.9847523620559342 
 V : 0.999999747460208 
MAE : 
 Alpha : 0.002689377882041364 
 Beta : 0.5084419454772547 
 V : 0.0017468670320417647 
MSE : 
 Alpha : 3.416202084707356e-05 
 Beta : 1.66547472079497 
 V : 5.1557921646603296e-05 
Max AERR : 
 V     0.080422
α     0.080375
β    10.100944
dtype: float64
Waktu yang dibutuhkan :  1.8341790239016216


In [99]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_31 = df_all_clean_30.drop(index = index_max_aerr)
print(len(df_all_clean_31))

6714


### AFTER REDUCE 31

In [100]:
x_all = df_all_clean_31.drop(columns = ["V","α","β"])
y_all = df_all_clean_31[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [101]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999825001711 
 Beta : 0.9859798471751942 
 V : 0.9999997646155153 
MAE : 
 Alpha : 0.002691839368961589 
 Beta : 0.4774067928830043 
 V : 0.0015705260699524965 
MSE : 
 Alpha : 2.16687801018428e-05 
 Beta : 1.541316361415488 
 V : 4.736723333802249e-05 
Max AERR : 
 V    0.085217
α    0.060339
β    9.863913
dtype: float64
Waktu yang dibutuhkan :  1.830320163567861


In [102]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_32 = df_all_clean_31.drop(index = index_max_aerr)
print(len(df_all_clean_32))

6713


### AFTER REDUCE 32

In [103]:
x_all = df_all_clean_32.drop(columns = ["V","α","β"])
y_all = df_all_clean_32[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [104]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999763082895 
 Beta : 0.9841078351906214 
 V : 0.9999997835298718 
MAE : 
 Alpha : 0.0029007883562725315 
 Beta : 0.4758546440726956 
 V : 0.001571810663628205 
MSE : 
 Alpha : 2.9704818063988463e-05 
 Beta : 1.713352362434301 
 V : 4.283382413809334e-05 
Max AERR : 
 V     0.074815
α     0.051164
β    13.349615
dtype: float64
Waktu yang dibutuhkan :  1.8470613797505697


In [105]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_33 = df_all_clean_32.drop(index = index_max_aerr)
print(len(df_all_clean_33))

6712


### AFTER REDUCE 33

In [106]:
x_all = df_all_clean_33.drop(columns = ["V","α","β"])
y_all = df_all_clean_33[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [107]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999820022719 
 Beta : 0.9819313671655014 
 V : 0.9999996527257021 
MAE : 
 Alpha : 0.0026030048566138596 
 Beta : 0.5209785858614318 
 V : 0.0021302722706457943 
MSE : 
 Alpha : 2.322633571794203e-05 
 Beta : 1.982663018887723 
 V : 6.827825013093455e-05 
Max AERR : 
 V     0.084053
α     0.052331
β    12.929680
dtype: float64
Waktu yang dibutuhkan :  1.8345162351926168


In [108]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_34 = df_all_clean_33.drop(index = index_max_aerr)
print(len(df_all_clean_34))

6711


### AFTER REDUCE 34

In [109]:
x_all = df_all_clean_34.drop(columns = ["V","α","β"])
y_all = df_all_clean_34[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [110]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999786509493 
 Beta : 0.9835457572319505 
 V : 0.9999997556613842 
MAE : 
 Alpha : 0.0027944258559907645 
 Beta : 0.502775417442728 
 V : 0.00170685162945919 
MSE : 
 Alpha : 2.7355361206612185e-05 
 Beta : 1.8924921238139776 
 V : 4.8748245576462514e-05 
Max AERR : 
 V     0.074711
α     0.057884
β    14.032261
dtype: float64
Waktu yang dibutuhkan :  1.8391971866289774


In [111]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_35 = df_all_clean_34.drop(index = index_max_aerr)
print(len(df_all_clean_35))

6710


### AFTER REDUCE 35

In [112]:
x_all = df_all_clean_35.drop(columns = ["V","α","β"])
y_all = df_all_clean_35[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [113]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.999999980198836 
 Beta : 0.9840160738974467 
 V : 0.9999997636338397 
MAE : 
 Alpha : 0.0028303457797262377 
 Beta : 0.5034851501969861 
 V : 0.0015917209804549776 
MSE : 
 Alpha : 2.5366837477890114e-05 
 Beta : 1.800420028828485 
 V : 4.7529516350077175e-05 
Max AERR : 
 V     0.074943
α     0.050238
β    11.976824
dtype: float64
Waktu yang dibutuhkan :  1.831849185625712


In [114]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_36 = df_all_clean_35.drop(index = index_max_aerr)
print(len(df_all_clean_36))

6709


### AFTER REDUCE 36

In [115]:
x_all = df_all_clean_36.drop(columns = ["V","α","β"])
y_all = df_all_clean_36[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [116]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999738340725 
 Beta : 0.9833821375989705 
 V : 0.9999997877667652 
MAE : 
 Alpha : 0.003326081960172426 
 Beta : 0.48062277834288347 
 V : 0.0016252762760998387 
MSE : 
 Alpha : 3.29070172602805e-05 
 Beta : 1.8197292153676679 
 V : 4.150479516016675e-05 
Max AERR : 
 V     0.075258
α     0.049462
β    11.821661
dtype: float64
Waktu yang dibutuhkan :  1.8351463596026103


In [117]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_37 = df_all_clean_36.drop(index = index_max_aerr)
print(len(df_all_clean_37))

6708


### AFTER REDUCE 37

In [118]:
x_all = df_all_clean_37.drop(columns = ["V","α","β"])
y_all = df_all_clean_37[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [119]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.999999984840712 
 Beta : 0.9815307925559213 
 V : 0.9999997375017241 
MAE : 
 Alpha : 0.0023519494322485693 
 Beta : 0.525959918225707 
 V : 0.0017709599486110052 
MSE : 
 Alpha : 1.8956466201636617e-05 
 Beta : 2.050883171098925 
 V : 5.1843585767290974e-05 
Max AERR : 
 V     0.074817
α     0.049335
β    14.448344
dtype: float64
Waktu yang dibutuhkan :  1.8419795433680217


In [120]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_38 = df_all_clean_37.drop(index = index_max_aerr)
print(len(df_all_clean_38))

6707


### AFTER REDUCE 38

In [121]:
x_all = df_all_clean_38.drop(columns = ["V","α","β"])
y_all = df_all_clean_38[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [122]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999797229634 
 Beta : 0.9834303172190553 
 V : 0.9999997368919491 
MAE : 
 Alpha : 0.0026780906334685976 
 Beta : 0.5128162890156647 
 V : 0.0018897118032941249 
MSE : 
 Alpha : 2.5352326420998613e-05 
 Beta : 1.8238039838509783 
 V : 5.19653287883851e-05 
Max AERR : 
 V     0.081435
α     0.048718
β    11.763475
dtype: float64
Waktu yang dibutuhkan :  1.8354536612828574


In [123]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_39 = df_all_clean_38.drop(index = index_max_aerr)
print(len(df_all_clean_39))

6706


### AFTER REDUCE 39

In [124]:
x_all = df_all_clean_39.drop(columns = ["V","α","β"])
y_all = df_all_clean_39[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [125]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999810644797 
 Beta : 0.9843119088968877 
 V : 0.999999707111458 
MAE : 
 Alpha : 0.00292930935887574 
 Beta : 0.5016120403003286 
 V : 0.0018457808773690784 
MSE : 
 Alpha : 2.3566231779547528e-05 
 Beta : 1.6872615745645418 
 V : 5.797790535189132e-05 
Max AERR : 
 V    0.081513
α    0.038228
β    9.558576
dtype: float64
Waktu yang dibutuhkan :  1.8381407618522645


In [126]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_40 = df_all_clean_39.drop(index = index_max_aerr)
print(len(df_all_clean_40))

6705


### AFTER REDUCE 40

In [127]:
x_all = df_all_clean_40.drop(columns = ["V","α","β"])
y_all = df_all_clean_40[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [128]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999743025513 
 Beta : 0.9821443111709135 
 V : 0.999999691323986 
MAE : 
 Alpha : 0.0034311681653425683 
 Beta : 0.5083332818473383 
 V : 0.0018407297923739262 
MSE : 
 Alpha : 3.203361078322819e-05 
 Beta : 1.9011435645342492 
 V : 6.09794882716634e-05 
Max AERR : 
 V     0.076190
α     0.039451
β    10.589069
dtype: float64
Waktu yang dibutuhkan :  1.8369463245073954


In [129]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_41 = df_all_clean_40.drop(index = index_max_aerr)
print(len(df_all_clean_41))

6704


### AFTER REDUCE 41

In [130]:
x_all = df_all_clean_41.drop(columns = ["V","α","β"])
y_all = df_all_clean_41[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [131]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999822537182 
 Beta : 0.9873651434594711 
 V : 0.9999997541977744 
MAE : 
 Alpha : 0.0023344237819430016 
 Beta : 0.4537950315498421 
 V : 0.0016357732679219568 
MSE : 
 Alpha : 2.233449706020762e-05 
 Beta : 1.4168638096644222 
 V : 4.928808519449712e-05 
Max AERR : 
 V     0.081151
α     0.054855
β    10.270374
dtype: float64
Waktu yang dibutuhkan :  1.829922858874003


In [132]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_42 = df_all_clean_41.drop(index = index_max_aerr)
print(len(df_all_clean_42))

6703


### AFTER REDUCE 42

In [133]:
x_all = df_all_clean_42.drop(columns = ["V","α","β"])
y_all = df_all_clean_42[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [134]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999781418617 
 Beta : 0.9878083703563936 
 V : 0.9999997089643333 
MAE : 
 Alpha : 0.002656073558848883 
 Beta : 0.4664964384661824 
 V : 0.0018980115051083538 
MSE : 
 Alpha : 2.7384304933077718e-05 
 Beta : 1.3658701639989814 
 V : 5.816420722638921e-05 
Max AERR : 
 V    0.079664
α    0.059284
β    9.302945
dtype: float64
Waktu yang dibutuhkan :  1.8279379288355508


In [135]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_43 = df_all_clean_42.drop(index = index_max_aerr)
print(len(df_all_clean_43))

6702


### AFTER REDUCE 43

In [136]:
x_all = df_all_clean_43.drop(columns = ["V","α","β"])
y_all = df_all_clean_43[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [137]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999770795356 
 Beta : 0.9835863663263281 
 V : 0.9999997386783672 
MAE : 
 Alpha : 0.0024739711936597824 
 Beta : 0.49830277382145155 
 V : 0.0017461861599062849 
MSE : 
 Alpha : 2.8554010329511736e-05 
 Beta : 1.7568578618611965 
 V : 5.2071114306191225e-05 
Max AERR : 
 V     0.083858
α     0.068155
β    11.893101
dtype: float64
Waktu yang dibutuhkan :  1.8360097050666808


In [138]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_44 = df_all_clean_43.drop(index = index_max_aerr)
print(len(df_all_clean_44))

6701


### AFTER REDUCE 44

In [139]:
x_all = df_all_clean_44.drop(columns = ["V","α","β"])
y_all = df_all_clean_44[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [140]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999862129972 
 Beta : 0.9838486020063449 
 V : 0.999999695824001 
MAE : 
 Alpha : 0.002295241830340103 
 Beta : 0.49253974536648526 
 V : 0.0019082321647309502 
MSE : 
 Alpha : 1.7626720497303956e-05 
 Beta : 1.7925254820284473 
 V : 6.102452212424785e-05 
Max AERR : 
 V     0.080020
α     0.051859
β    12.402461
dtype: float64
Waktu yang dibutuhkan :  1.8306045214335123


In [141]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_45 = df_all_clean_44.drop(index = index_max_aerr)
print(len(df_all_clean_45))

6700


### AFTER REDUCE 45

In [142]:
x_all = df_all_clean_45.drop(columns = ["V","α","β"])
y_all = df_all_clean_45[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [143]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999882841694 
 Beta : 0.9858857262847728 
 V : 0.999999763029445 
MAE : 
 Alpha : 0.0019901448821390997 
 Beta : 0.484551620603368 
 V : 0.0015754310434314596 
MSE : 
 Alpha : 1.4979024203481863e-05 
 Beta : 1.5734467513349844 
 V : 4.76332765424633e-05 
Max AERR : 
 V     0.074849
α     0.039299
β    10.423491
dtype: float64
Waktu yang dibutuhkan :  1.8356659730275473


In [144]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_46 = df_all_clean_45.drop(index = index_max_aerr)
print(len(df_all_clean_46))

6699


### AFTER REDUCE 46

In [145]:
x_all = df_all_clean_46.drop(columns = ["V","α","β"])
y_all = df_all_clean_46[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [146]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999673448308 
 Beta : 0.9835118830896732 
 V : 0.9999997702236337 
MAE : 
 Alpha : 0.0025089832384530375 
 Beta : 0.4979029150945871 
 V : 0.00154842911984076 
MSE : 
 Alpha : 4.220049381427945e-05 
 Beta : 1.8239368695061675 
 V : 4.63460707983565e-05 
Max AERR : 
 V     0.080961
α     0.071318
β    11.778419
dtype: float64
Waktu yang dibutuhkan :  1.8320097168286642


In [147]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_47 = df_all_clean_46.drop(index = index_max_aerr)
print(len(df_all_clean_47))

6698


### AFTER REDUCE 47

In [148]:
x_all = df_all_clean_47.drop(columns = ["V","α","β"])
y_all = df_all_clean_47[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [149]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999869104678 
 Beta : 0.9861414636337292 
 V : 0.9999997259135626 
MAE : 
 Alpha : 0.002067391307369238 
 Beta : 0.475328824620637 
 V : 0.0015883403397315185 
MSE : 
 Alpha : 1.642420853615022e-05 
 Beta : 1.5248578976284304 
 V : 5.5021711036523626e-05 
Max AERR : 
 V    0.111838
α    0.053666
β    9.774082
dtype: float64
Waktu yang dibutuhkan :  1.824476667245229


In [150]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_48 = df_all_clean_47.drop(index = index_max_aerr)
print(len(df_all_clean_48))

6697


### AFTER REDUCE 48

In [151]:
x_all = df_all_clean_48.drop(columns = ["V","α","β"])
y_all = df_all_clean_48[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [152]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999791805648 
 Beta : 0.9873685035908556 
 V : 0.9999998024412914 
MAE : 
 Alpha : 0.002997647550987879 
 Beta : 0.44296020624164456 
 V : 0.001492654476972445 
MSE : 
 Alpha : 2.6096207456495595e-05 
 Beta : 1.388734931378668 
 V : 3.9702969640764844e-05 
Max AERR : 
 V    0.081564
α    0.047327
β    9.645204
dtype: float64
Waktu yang dibutuhkan :  1.815116024017334


In [153]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_49 = df_all_clean_48.drop(index = index_max_aerr)
print(len(df_all_clean_49))

6696


### AFTER REDUCE 49

In [154]:
x_all = df_all_clean_49.drop(columns = ["V","α","β"])
y_all = df_all_clean_49[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [155]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999791953348 
 Beta : 0.9867675862312114 
 V : 0.9999998030464354 
MAE : 
 Alpha : 0.002844769642394002 
 Beta : 0.4835766929532452 
 V : 0.0015882510723763483 
MSE : 
 Alpha : 2.6063898264081343e-05 
 Beta : 1.4393844376440033 
 V : 3.941940559266608e-05 
Max AERR : 
 V    0.075182
α    0.056576
β    9.439480
dtype: float64
Waktu yang dibutuhkan :  1.8296287457148235


In [156]:
y_pred_df = pd.DataFrame(y0_pred, columns = ["V","α","β"], index = y0_test.index)
aerr = np.abs(y_pred_df - y0_test)
index_max_aerr = aerr["β"].idxmax()
df_all_clean_50 = df_all_clean_49.drop(index = index_max_aerr)
print(len(df_all_clean_50))

6695


### AFTER REDUCE 50

In [157]:
x_all = df_all_clean_50.drop(columns = ["V","α","β"])
y_all = df_all_clean_50[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [158]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999794497764 
 Beta : 0.9886444493329439 
 V : 0.9999998349337534 
MAE : 
 Alpha : 0.0028038884943813696 
 Beta : 0.44509165187523836 
 V : 0.001484830621008814 
MSE : 
 Alpha : 2.603164413458015e-05 
 Beta : 1.2754112001993834 
 V : 3.3499555328369416e-05 
Max AERR : 
 V    0.047681
α    0.050894
β    9.078520
dtype: float64
Waktu yang dibutuhkan :  1.8254061381022135


# SAVE NEW DATASET

In [159]:
df_all_clean_15.to_csv('data_clean_reduce_15_z_score_3_sec.csv', index = False)

In [160]:
df_all_clean_21.to_csv('data_clean_reduce_21_z_score_3_sec.csv', index = False)

In [161]:
df_all_clean_23.to_csv('data_clean_reduce_23_z_score_3_sec.csv', index = False)

In [162]:
df_all_clean_24.to_csv('data_clean_reduce_24_z_score_3_sec.csv', index = False)

In [163]:
df_all_clean_25.to_csv('data_clean_reduce_25_z_score_3_sec.csv', index = False)

In [164]:
df_all_clean_26.to_csv('data_clean_reduce_26_z_score_3_sec.csv', index = False)

In [165]:
df_all_clean_27.to_csv('data_clean_reduce_27_z_score_3_sec.csv', index = False)

In [168]:
df_all_clean_31.to_csv('data_clean_reduce_31_z_score_3_sec.csv', index = False)

In [169]:
df_all_clean_36.to_csv('data_clean_reduce_36_z_score_3_sec.csv', index = False)

In [170]:
df_all_clean_41.to_csv('data_clean_reduce_41_z_score_3_sec.csv', index = False)

In [171]:
df_all_clean_42.to_csv('data_clean_reduce_42_z_score_3_sec.csv', index = False)

In [172]:
df_all_clean_43.to_csv('data_clean_reduce_43_z_score_3_sec.csv', index = False)

In [173]:
df_all_clean_44.to_csv('data_clean_reduce_44_z_score_3_sec.csv', index = False)

In [174]:
df_all_clean_45.to_csv('data_clean_reduce_45_z_score_3_sec.csv', index = False)

In [175]:
df_all_clean_46.to_csv('data_clean_reduce_46_z_score_3_sec.csv', index = False)

In [176]:
df_all_clean_47.to_csv('data_clean_reduce_47_z_score_3_sec.csv', index = False)

In [177]:
df_all_clean_48.to_csv('data_clean_reduce_48_z_score_3_sec.csv', index = False)

In [178]:
df_all_clean_49.to_csv('data_clean_reduce_49_z_score_3_sec.csv', index = False)

In [179]:
df_all_clean_50.to_csv('data_clean_reduce_50_z_score_3_sec.csv', index = False)

In [181]:
df1 = pd.read_csv("z_score_3_data_all.csv")
df2 = pd.read_csv("data_clean_reduce_15_z_score_3_sec.csv")
missing_indices = df1.index.difference(df2.index)
missing_rows = df1.merge(df2, how='outer', indicator=True).query('_merge == "left_only"')
print(missing_rows)
print("Reduce index:", missing_rows.index.tolist())

          Pβ2      Pα1      Pβ1      Pα2       P3          V   α   β  \
3077 -0.37203 -0.13467 -0.36698 -0.27608 -0.37708  20.027975  90 -20   
3078 -0.36361 -0.13467 -0.35856 -0.27271 -0.37540  20.027975  90 -20   
3079 -0.37540 -0.13131 -0.36866 -0.26934 -0.37540  20.027975  90 -20   
3081 -0.37372 -0.13131 -0.36866 -0.27103 -0.38045  20.027975  90 -20   
4437 -0.85853 -0.18517 -0.79457 -0.59592 -0.83160  30.000000  85 -20   
4469 -0.82823 -0.18517 -0.81140 -0.60939 -0.83160  30.024409  85   0   
4472 -0.83328 -0.18517 -0.83497 -0.61107 -0.82992  30.024409  85   0   
4473 -0.83328 -0.18517 -0.84170 -0.60939 -0.82150  30.024409  85   0   
4475 -0.84170 -0.18686 -0.84507 -0.58414 -0.82655  30.024409  85   0   
5853 -1.49654 -0.34341 -1.49654 -1.03361 -1.48476  40.037530  85  -5   
5857 -1.48981 -0.34341 -1.48308 -1.03024 -1.48139  40.037530  85  -5   
5859 -1.50833 -0.34341 -1.49486 -1.04708 -1.48139  40.037530  85  -5   
7239 -2.33656 -0.51344 -2.36013 -1.60260 -2.34329  50.031069  85

In [182]:
df1 = pd.read_csv("z_score_3_data_all.csv")
df2 = pd.read_csv("data_clean_reduce_21_z_score_3_sec.csv")
missing_indices = df1.index.difference(df2.index)
missing_rows = df1.merge(df2, how='outer', indicator=True).query('_merge == "left_only"')
print(missing_rows)
print("Reduce index:", missing_rows.index.tolist())

          Pβ2      Pα1      Pβ1      Pα2       P3          V   α   β  \
3077 -0.37203 -0.13467 -0.36698 -0.27608 -0.37708  20.027975  90 -20   
3078 -0.36361 -0.13467 -0.35856 -0.27271 -0.37540  20.027975  90 -20   
3079 -0.37540 -0.13131 -0.36866 -0.26934 -0.37540  20.027975  90 -20   
3080 -0.36866 -0.13131 -0.36530 -0.26934 -0.37372  20.027975  90 -20   
3081 -0.37372 -0.13131 -0.36866 -0.27103 -0.38045  20.027975  90 -20   
3117 -0.35688 -0.13131 -0.35351 -0.26934 -0.37035  20.019834  90   0   
3121 -0.36866 -0.13131 -0.37203 -0.27608 -0.38045  20.019834  90   0   
4437 -0.85853 -0.18517 -0.79457 -0.59592 -0.83160  30.000000  85 -20   
4469 -0.82823 -0.18517 -0.81140 -0.60939 -0.83160  30.024409  85   0   
4472 -0.83328 -0.18517 -0.83497 -0.61107 -0.82992  30.024409  85   0   
4473 -0.83328 -0.18517 -0.84170 -0.60939 -0.82150  30.024409  85   0   
4475 -0.84170 -0.18686 -0.84507 -0.58414 -0.82655  30.024409  85   0   
5853 -1.49654 -0.34341 -1.49654 -1.03361 -1.48476  40.037530  85

In [183]:
df1 = pd.read_csv("z_score_3_data_all.csv")
df2 = pd.read_csv("data_clean_reduce_23_z_score_3_sec.csv")
missing_indices = df1.index.difference(df2.index)
missing_rows = df1.merge(df2, how='outer', indicator=True).query('_merge == "left_only"')
print(missing_rows)
print("Reduce index:", missing_rows.index.tolist())

          Pβ2      Pα1      Pβ1      Pα2       P3          V   α   β  \
3013 -0.36361 -0.02525 -0.38213 -0.28113 -0.34846  20.006058  80   0   
3017 -0.36193 -0.02862 -0.37203 -0.28113 -0.34846  20.006058  80   0   
3077 -0.37203 -0.13467 -0.36698 -0.27608 -0.37708  20.027975  90 -20   
3078 -0.36361 -0.13467 -0.35856 -0.27271 -0.37540  20.027975  90 -20   
3079 -0.37540 -0.13131 -0.36866 -0.26934 -0.37540  20.027975  90 -20   
3080 -0.36866 -0.13131 -0.36530 -0.26934 -0.37372  20.027975  90 -20   
3081 -0.37372 -0.13131 -0.36866 -0.27103 -0.38045  20.027975  90 -20   
3117 -0.35688 -0.13131 -0.35351 -0.26934 -0.37035  20.019834  90   0   
3121 -0.36866 -0.13131 -0.37203 -0.27608 -0.38045  20.019834  90   0   
4437 -0.85853 -0.18517 -0.79457 -0.59592 -0.83160  30.000000  85 -20   
4469 -0.82823 -0.18517 -0.81140 -0.60939 -0.83160  30.024409  85   0   
4472 -0.83328 -0.18517 -0.83497 -0.61107 -0.82992  30.024409  85   0   
4473 -0.83328 -0.18517 -0.84170 -0.60939 -0.82150  30.024409  85

In [184]:
df1 = pd.read_csv("z_score_3_data_all.csv")
df2 = pd.read_csv("data_clean_reduce_24_z_score_3_sec.csv")
missing_indices = df1.index.difference(df2.index)
missing_rows = df1.merge(df2, how='outer', indicator=True).query('_merge == "left_only"')
print(missing_rows)
print("Reduce index:", missing_rows.index.tolist())

          Pβ2      Pα1      Pβ1      Pα2       P3          V   α   β  \
3013 -0.36361 -0.02525 -0.38213 -0.28113 -0.34846  20.006058  80   0   
3017 -0.36193 -0.02862 -0.37203 -0.28113 -0.34846  20.006058  80   0   
3029 -0.37708 -0.08080 -0.38045 -0.27271 -0.37035  20.037314  85 -20   
3077 -0.37203 -0.13467 -0.36698 -0.27608 -0.37708  20.027975  90 -20   
3078 -0.36361 -0.13467 -0.35856 -0.27271 -0.37540  20.027975  90 -20   
3079 -0.37540 -0.13131 -0.36866 -0.26934 -0.37540  20.027975  90 -20   
3080 -0.36866 -0.13131 -0.36530 -0.26934 -0.37372  20.027975  90 -20   
3081 -0.37372 -0.13131 -0.36866 -0.27103 -0.38045  20.027975  90 -20   
3117 -0.35688 -0.13131 -0.35351 -0.26934 -0.37035  20.019834  90   0   
3121 -0.36866 -0.13131 -0.37203 -0.27608 -0.38045  20.019834  90   0   
4437 -0.85853 -0.18517 -0.79457 -0.59592 -0.83160  30.000000  85 -20   
4469 -0.82823 -0.18517 -0.81140 -0.60939 -0.83160  30.024409  85   0   
4472 -0.83328 -0.18517 -0.83497 -0.61107 -0.82992  30.024409  85

In [185]:
df1 = pd.read_csv("z_score_3_data_all.csv")
df2 = pd.read_csv("data_clean_reduce_25_z_score_3_sec.csv")
missing_indices = df1.index.difference(df2.index)
missing_rows = df1.merge(df2, how='outer', indicator=True).query('_merge == "left_only"')
print(missing_rows)
print("Reduce index:", missing_rows.index.tolist())

          Pβ2      Pα1      Pβ1      Pα2       P3          V   α   β  \
3013 -0.36361 -0.02525 -0.38213 -0.28113 -0.34846  20.006058  80   0   
3017 -0.36193 -0.02862 -0.37203 -0.28113 -0.34846  20.006058  80   0   
3029 -0.37708 -0.08080 -0.38045 -0.27271 -0.37035  20.037314  85 -20   
3077 -0.37203 -0.13467 -0.36698 -0.27608 -0.37708  20.027975  90 -20   
3078 -0.36361 -0.13467 -0.35856 -0.27271 -0.37540  20.027975  90 -20   
3079 -0.37540 -0.13131 -0.36866 -0.26934 -0.37540  20.027975  90 -20   
3080 -0.36866 -0.13131 -0.36530 -0.26934 -0.37372  20.027975  90 -20   
3081 -0.37372 -0.13131 -0.36866 -0.27103 -0.38045  20.027975  90 -20   
3117 -0.35688 -0.13131 -0.35351 -0.26934 -0.37035  20.019834  90   0   
3121 -0.36866 -0.13131 -0.37203 -0.27608 -0.38045  20.019834  90   0   
4437 -0.85853 -0.18517 -0.79457 -0.59592 -0.83160  30.000000  85 -20   
4444 -0.87200 -0.18349 -0.84338 -0.60771 -0.83160  29.990107  85 -15   
4469 -0.82823 -0.18517 -0.81140 -0.60939 -0.83160  30.024409  85

In [186]:
df1 = pd.read_csv("z_score_3_data_all.csv")
df2 = pd.read_csv("data_clean_reduce_26_z_score_3_sec.csv")
missing_indices = df1.index.difference(df2.index)
missing_rows = df1.merge(df2, how='outer', indicator=True).query('_merge == "left_only"')
print(missing_rows)
print("Reduce index:", missing_rows.index.tolist())

          Pβ2      Pα1      Pβ1      Pα2       P3          V   α   β  \
3013 -0.36361 -0.02525 -0.38213 -0.28113 -0.34846  20.006058  80   0   
3017 -0.36193 -0.02862 -0.37203 -0.28113 -0.34846  20.006058  80   0   
3029 -0.37708 -0.08080 -0.38045 -0.27271 -0.37035  20.037314  85 -20   
3077 -0.37203 -0.13467 -0.36698 -0.27608 -0.37708  20.027975  90 -20   
3078 -0.36361 -0.13467 -0.35856 -0.27271 -0.37540  20.027975  90 -20   
3079 -0.37540 -0.13131 -0.36866 -0.26934 -0.37540  20.027975  90 -20   
3080 -0.36866 -0.13131 -0.36530 -0.26934 -0.37372  20.027975  90 -20   
3081 -0.37372 -0.13131 -0.36866 -0.27103 -0.38045  20.027975  90 -20   
3117 -0.35688 -0.13131 -0.35351 -0.26934 -0.37035  20.019834  90   0   
3121 -0.36866 -0.13131 -0.37203 -0.27608 -0.38045  20.019834  90   0   
4437 -0.85853 -0.18517 -0.79457 -0.59592 -0.83160  30.000000  85 -20   
4444 -0.87200 -0.18349 -0.84338 -0.60771 -0.83160  29.990107  85 -15   
4467 -0.83833 -0.19022 -0.84675 -0.60602 -0.84002  30.065874  85

In [187]:
df1 = pd.read_csv("z_score_3_data_all.csv")
df2 = pd.read_csv("data_clean_reduce_27_z_score_3_sec.csv")
missing_indices = df1.index.difference(df2.index)
missing_rows = df1.merge(df2, how='outer', indicator=True).query('_merge == "left_only"')
print(missing_rows)
print("Reduce index:", missing_rows.index.tolist())

          Pβ2      Pα1      Pβ1      Pα2       P3          V   α   β  \
3013 -0.36361 -0.02525 -0.38213 -0.28113 -0.34846  20.006058  80   0   
3017 -0.36193 -0.02862 -0.37203 -0.28113 -0.34846  20.006058  80   0   
3029 -0.37708 -0.08080 -0.38045 -0.27271 -0.37035  20.037314  85 -20   
3073 -0.36698 -0.13299 -0.37035 -0.26934 -0.37708  20.027975  90 -20   
3077 -0.37203 -0.13467 -0.36698 -0.27608 -0.37708  20.027975  90 -20   
3078 -0.36361 -0.13467 -0.35856 -0.27271 -0.37540  20.027975  90 -20   
3079 -0.37540 -0.13131 -0.36866 -0.26934 -0.37540  20.027975  90 -20   
3080 -0.36866 -0.13131 -0.36530 -0.26934 -0.37372  20.027975  90 -20   
3081 -0.37372 -0.13131 -0.36866 -0.27103 -0.38045  20.027975  90 -20   
3117 -0.35688 -0.13131 -0.35351 -0.26934 -0.37035  20.019834  90   0   
3121 -0.36866 -0.13131 -0.37203 -0.27608 -0.38045  20.019834  90   0   
4437 -0.85853 -0.18517 -0.79457 -0.59592 -0.83160  30.000000  85 -20   
4444 -0.87200 -0.18349 -0.84338 -0.60771 -0.83160  29.990107  85

In [195]:
df1 = pd.read_csv("z_score_3_data_all.csv")
df2 = pd.read_csv("data_clean_reduce_31_z_score_3_sec.csv")
missing_indices = df1.index.difference(df2.index)
missing_rows = df1.merge(df2, how='outer', indicator=True).query('_merge == "left_only"')
print(missing_rows)
print("Reduce index:", missing_rows.index.tolist())

          Pβ2      Pα1      Pβ1      Pα2       P3          V   α   β  \
3013 -0.36361 -0.02525 -0.38213 -0.28113 -0.34846  20.006058  80   0   
3017 -0.36193 -0.02862 -0.37203 -0.28113 -0.34846  20.006058  80   0   
3029 -0.37708 -0.08080 -0.38045 -0.27271 -0.37035  20.037314  85 -20   
3073 -0.36698 -0.13299 -0.37035 -0.26934 -0.37708  20.027975  90 -20   
3077 -0.37203 -0.13467 -0.36698 -0.27608 -0.37708  20.027975  90 -20   
3078 -0.36361 -0.13467 -0.35856 -0.27271 -0.37540  20.027975  90 -20   
3079 -0.37540 -0.13131 -0.36866 -0.26934 -0.37540  20.027975  90 -20   
3080 -0.36866 -0.13131 -0.36530 -0.26934 -0.37372  20.027975  90 -20   
3081 -0.37372 -0.13131 -0.36866 -0.27103 -0.38045  20.027975  90 -20   
3117 -0.35688 -0.13131 -0.35351 -0.26934 -0.37035  20.019834  90   0   
3121 -0.36866 -0.13131 -0.37203 -0.27608 -0.38045  20.019834  90   0   
4437 -0.85853 -0.18517 -0.79457 -0.59592 -0.83160  30.000000  85 -20   
4444 -0.87200 -0.18349 -0.84338 -0.60771 -0.83160  29.990107  85

In [196]:
df1 = pd.read_csv("z_score_3_data_all.csv")
df2 = pd.read_csv("data_clean_reduce_27_z_score_3_sec.csv")
missing_indices = df1.index.difference(df2.index)
missing_rows = df1.merge(df2, how='outer', indicator=True).query('_merge == "left_only"')
print(missing_rows)
print("Reduce index:", missing_rows.index.tolist())

          Pβ2      Pα1      Pβ1      Pα2       P3          V   α   β  \
3013 -0.36361 -0.02525 -0.38213 -0.28113 -0.34846  20.006058  80   0   
3017 -0.36193 -0.02862 -0.37203 -0.28113 -0.34846  20.006058  80   0   
3029 -0.37708 -0.08080 -0.38045 -0.27271 -0.37035  20.037314  85 -20   
3073 -0.36698 -0.13299 -0.37035 -0.26934 -0.37708  20.027975  90 -20   
3077 -0.37203 -0.13467 -0.36698 -0.27608 -0.37708  20.027975  90 -20   
3078 -0.36361 -0.13467 -0.35856 -0.27271 -0.37540  20.027975  90 -20   
3079 -0.37540 -0.13131 -0.36866 -0.26934 -0.37540  20.027975  90 -20   
3080 -0.36866 -0.13131 -0.36530 -0.26934 -0.37372  20.027975  90 -20   
3081 -0.37372 -0.13131 -0.36866 -0.27103 -0.38045  20.027975  90 -20   
3117 -0.35688 -0.13131 -0.35351 -0.26934 -0.37035  20.019834  90   0   
3121 -0.36866 -0.13131 -0.37203 -0.27608 -0.38045  20.019834  90   0   
4437 -0.85853 -0.18517 -0.79457 -0.59592 -0.83160  30.000000  85 -20   
4444 -0.87200 -0.18349 -0.84338 -0.60771 -0.83160  29.990107  85

In [191]:
df1[(df1['V'] >= 30) & (df1['V'] <= 31) & (df1['α'] == 85) & (df1['β'] == 0)]
'''
4469 -0.82823 -0.18517 -0.81140 -0.60939 -0.83160  30.024409  85   0   
4472 -0.83328 -0.18517 -0.83497 -0.61107 -0.82992  30.024409  85   0   
4473 -0.83328 -0.18517 -0.84170 -0.60939 -0.82150  30.024409  85   0   
4475 -0.84170 -0.18686 -0.84507 -0.58414 -0.82655  30.024409  85   0   
4476 -0.85853 -0.19022 -0.85853 -0.60266 -0.83160  30.024409  85   0   
4477 -0.85180 -0.18854 -0.85685 -0.61107 -0.82823  30.024409  85   0   
4478 -0.85012 -0.18517 -0.84170 -0.60771 -0.83328  30.024409  85   0 
'''
'''
780	-0.82823	-0.18517	-0.81140	-0.60939	-0.83160	30.024409	85	0
783	-0.83328	-0.18517	-0.83497	-0.61107	-0.82992	30.024409	85	0
784	-0.83328	-0.18517	-0.84170	-0.60939	-0.82150	30.024409	85	0
786	-0.84170	-0.18686	-0.84507	-0.58414	-0.82655	30.024409	85	0
787	-0.85853	-0.19022	-0.85853	-0.60266	-0.83160	30.024409	85	0
788	-0.85180	-0.18854	-0.85685	-0.61107	-0.82823	30.024409	85	0
789	-0.85012	-0.18517	-0.84170	-0.60771	-0.83328	30.024409	85	0
'''
'''
   Pβ2	       Pα1	        Pβ1	       Pα2	        P3	        V	    α	β
-0.86358	-0.18349	-0.83665	-0.61107	-0.83328	30.024409	85	0
-0.86022	-0.18517	-0.83833	-0.61949	-0.83665	30.024409	85	0
-0.87369	-0.18854	-0.82318	-0.60939	-0.83328	30.024409	85	0
'''

,Pβ2,Pα1,Pβ1,Pα2,P3,V,α,β
780,-0.82823,-0.18517,-0.81140,-0.60939,-0.83160,30.024409,85,0
781,-0.86358,-0.18349,-0.83665,-0.61107,-0.83328,30.024409,85,0
782,-0.86022,-0.18517,-0.83833,-0.61949,-0.83665,30.024409,85,0
783,-0.83328,-0.18517,-0.83497,-0.61107,-0.82992,30.024409,85,0
784,-0.83328,-0.18517,-0.84170,-0.60939,-0.82150,30.024409,85,0
785,-0.87369,-0.18854,-0.82318,-0.60939,-0.83328,30.024409,85,0
786,-0.84170,-0.18686,-0.84507,-0.58414,-0.82655,30.024409,85,0
787,-0.85853,-0.19022,-0.85853,-0.60266,-0.83160,30.024409,85,0
788,-0.85180,-0.18854,-0.85685,-0.61107,-0.82823,30.024409,85,0
789,-0.85012,-0.18517,-0.84170,-0.60771,-0.83328,30.024409,85,0


In [178]:
df1 = pd.read_csv("data_all.csv")
df2 = pd.read_csv("z_score_3_data_all.csv")
missing_indices = df1.index.difference(df2.index)
missing_rows = df1.merge(df2, how='outer', indicator=True).query('_merge == "left_only"')
print(missing_rows)
print("Reduce index:", missing_rows.index.tolist())

          Pβ2      Pα1      Pβ1      Pα2       P3          V   α   β  \
1627 -0.80803  1.26087 -1.24908 -1.78272 -0.23568  50.012185  45  -5   
1757 -0.13636  0.21716 -0.18854 -0.26598 -0.03030  20.115891  45  -5   
2503  0.36361  0.00337 -0.27271  0.00000  0.80467  40.064769   0 -10   
4629 -0.21043  0.40738 -0.55889 -0.64306 -0.14309  30.064446  45 -20   
6366  0.02862  0.01852  0.02693  0.04545  0.46799  30.045805   0   0   

         _merge  
1627  left_only  
1757  left_only  
2503  left_only  
4629  left_only  
6366  left_only  
Reduce index: [1627, 1757, 2503, 4629, 6366]


In [179]:
df1 = pd.read_csv("data_all.csv")
df2 = pd.read_csv("z_score_2_9_data_all.csv")
missing_indices = df1.index.difference(df2.index)
missing_rows = df1.merge(df2, how='outer', indicator=True).query('_merge == "left_only"')
print(missing_rows)
print("Reduce index:", missing_rows.index.tolist())

          Pβ2      Pα1      Pβ1      Pα2       P3          V   α   β  \
327  -0.34005  0.47472 -0.37035 -0.58414 -0.05219  29.986933  45   0   
1627 -0.80803  1.26087 -1.24908 -1.78272 -0.23568  50.012185  45  -5   
1757 -0.13636  0.21716 -0.18854 -0.26598 -0.03030  20.115891  45  -5   
2276  0.09090  0.00673 -0.06229  0.00337  0.19864  20.065718   0 -10   
2382 -0.11952  0.20537 -0.20874 -0.26766 -0.04545  20.034440  45 -10   
2503  0.36361  0.00337 -0.27271  0.00000  0.80467  40.064769   0 -10   
4629 -0.21043  0.40738 -0.55889 -0.64306 -0.14309  30.064446  45 -20   
6366  0.02862  0.01852  0.02693  0.04545  0.46799  30.045805   0   0   

         _merge  
327   left_only  
1627  left_only  
1757  left_only  
2276  left_only  
2382  left_only  
2503  left_only  
4629  left_only  
6366  left_only  
Reduce index: [327, 1627, 1757, 2276, 2382, 2503, 4629, 6366]


In [180]:
print(len(df2))

6742


In [166]:
df1.iloc[3065:3090]

,Pβ2,Pα1,Pβ1,Pα2,P3,V,α,β
3065,0.03198,0.01852,0.02525,0.03703,0.46967,30.028268,0,0
3066,0.03367,0.02188,0.03030,0.04040,0.47472,30.028268,0,0
3067,0.03198,0.02020,0.02693,0.04040,0.47472,30.028268,0,0
3068,0.03198,0.02020,0.02357,0.03872,0.47304,30.028268,0,0
3069,0.03198,0.01683,0.02357,0.04882,0.46799,30.045805,0,0
3070,0.03198,0.01852,0.02357,0.04714,0.46799,30.045805,0,0
3071,0.03198,0.01683,0.02693,0.04545,0.46799,30.045805,0,0
3072,0.03198,0.01683,0.02693,0.04545,0.46799,30.045805,0,0
3073,0.03367,0.01683,0.02693,0.04545,0.46967,30.045805,0,0
3074,0.03367,0.01852,0.02525,0.04545,0.46799,30.045805,0,0


In [170]:
df1.iloc[3110:3130]

,Pβ2,Pα1,Pβ1,Pα2,P3,V,α,β
3110,0.27776,0.07070,-0.23231,-0.10269,0.41243,29.976117,5,-15
3111,0.27944,0.07239,-0.23399,-0.10269,0.41412,29.976117,5,-15
3112,0.27944,0.07070,-0.23063,-0.10100,0.41412,29.976117,5,-15
3113,0.27944,0.07070,-0.23231,-0.10100,0.41412,29.976117,5,-15
3114,0.27944,0.07070,-0.23568,-0.10437,0.41412,29.976117,5,-15
3115,0.27776,0.07070,-0.23231,-0.10437,0.41243,29.976117,5,-15
3116,0.27944,0.07070,-0.23231,-0.10437,0.41243,29.976117,5,-15
3117,0.28113,0.07239,-0.23231,-0.10100,0.41412,29.976117,5,-15
3118,0.20201,0.09595,-0.15319,-0.08417,0.44273,30.041130,5,-10
3119,0.20201,0.09764,-0.15319,-0.08417,0.44273,30.041130,5,-10


In [173]:
df1.iloc[4460:4490]

,Pβ2,Pα1,Pβ1,Pα2,P3,V,α,β
4460,0.49492,0.11784,-0.41917,-0.19864,0.74406,40.067981,5,-15
4461,0.49492,0.11784,-0.41748,-0.19696,0.74238,40.067981,5,-15
4462,0.49492,0.12120,-0.41412,-0.19527,0.74575,40.067981,5,-15
4463,0.49492,0.12289,-0.41917,-0.19696,0.74406,40.067981,5,-15
4464,0.49492,0.12120,-0.41917,-0.19696,0.74575,40.067981,5,-15
4465,0.49492,0.11952,-0.42085,-0.19527,0.74575,40.067981,5,-15
4466,0.35351,0.16666,-0.28449,-0.16161,0.79457,40.048304,5,-10
4467,0.35351,0.16666,-0.28618,-0.16161,0.79457,40.048304,5,-10
4468,0.35351,0.16834,-0.28449,-0.16329,0.79625,40.048304,5,-10
4469,0.35351,0.16666,-0.28281,-0.16329,0.79625,40.048304,5,-10


IndexError: single positional indexer is out-of-bounds